In [ ]:
%pip install -q "git+https://github.com/AI4Finance-Foundation/FinRL.git@master" 

In [ ]:
%pip install pyfolio
%pip install pandas

%pip install matplotlib
%pip install warnings
%pip install "numpy<2.0"

In [ ]:
%matplotlib inline

from finrl.plot import convert_daily_return_to_pyfolio_ts
from pyfolio import timeseries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from finrl.plot import backtest_stats, get_daily_return, get_baseline,convert_daily_return_to_pyfolio_ts




In [ ]:
train_start_date = "2010-01-01"
train_end_date = "2020-12-31"

validate_start_date = "2021-01-01"
validate_end_date = "2022-12-31"

trade_start_date = "2023-01-01"
trade_end_date = "2025-11-01"


baseline_df = get_baseline(
        ticker="^DJI", 
        start = trade_start_date,
        end =  trade_end_date)


baseline_returns = get_daily_return(baseline_df, value_col_name="close")
baseline_df_stats = backtest_stats(baseline_df, value_col_name = 'close')
baseline_returns.to_csv("dji_baseline_daily_returns.csv")



In [ ]:
models = {
    "DDPG": "ddpg__performance.csv",
    "PPO": "ppo_performance.csv",
    "TD3": "td3_performance.csv",
    "DDPG_opt": "ddpg_Opt_performance.csv",
    "PPO_opt": "ppo_Opt_performance.csv",
    "TD3_opt": "td3_Opt_performance.csv",
    "1/N": "equal_weight_daily_returns.csv",
    "MSRP": "markowitz_daily_returns.csv"}

metriken_storage = {}
returns_storage = {}
metrics = timeseries.perf_stats

for model, file in models.items():
    df = pd.read_csv(file)
    returns_ts = convert_daily_return_to_pyfolio_ts(df)
    kennzahlen = metrics(returns=returns_ts, factor_returns=baseline_returns, positions=None, transactions=None, turnover_denom="AGB")
    metriken_storage[model] = kennzahlen
    returns_storage[model] = returns_ts




In [ ]:
%matplotlib inline
import seaborn as sns

sns.set_style("whitegrid")
sns.set_palette("colorblind")

baseline_kennzahlen = timeseries.perf_stats(
    returns=baseline_returns, 
    factor_returns=baseline_returns, 
    positions=None, 
    transactions=None, 
    turnover_denom="AGB"
)
metriken_storage['DJIA'] = baseline_kennzahlen

metriken_df = pd.DataFrame(metriken_storage)

wichtige_metriken = ['Annual return', 'Cumulative returns', 'Annual volatility', 
                     'Sharpe ratio', 'Calmar ratio', 'Max drawdown']

metrik_namen_deutsch = {
    'Annual return': 'Jährliche Rendite',
    'Cumulative returns': 'Kumulative Rendite',
    'Annual volatility': 'Jährliche Volatilität',
    'Sharpe ratio': 'Sharpe Ratio',
    'Calmar ratio': 'Calmar Ratio',
    'Max drawdown': 'Max drawdown'
}

farben = list(sns.color_palette("colorblind", n_colors=len(metriken_df.columns) - 1))
farben.append((0, 0, 0))

fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

for idx, metrik in enumerate(wichtige_metriken):
    ax = axes[idx]
    
    werte = metriken_df.loc[metrik]
    
    sns.barplot(x=list(werte.index), y=list(werte.values), palette=farben, ax=ax)
    
    for i, v in enumerate(werte.values):
        ax.text(i, v + 0.01 * abs(v), f'{v:.3f}', ha='center', fontsize=9, fontweight='bold')
    
    ax.set_title(metrik_namen_deutsch[metrik], fontsize=13, fontweight='bold')
    ax.set_xlabel('')
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('metriken_vergleich.pdf', dpi=300, bbox_inches='tight')
plt.show()

print("\n" + "="*50)
print("RANKING nach Sharpe Ratio:")
print("="*50)
ranking = metriken_df.loc['Sharpe ratio'].sort_values(ascending=False)
for i, (model, wert) in enumerate(ranking.items(), 1):
    print(f"  {i}. {model}: {wert:.4f}")


In [ ]:
%matplotlib inline
import seaborn as sns

sns.set_style("whitegrid")
farben = sns.color_palette("colorblind", n_colors=len(returns_storage))

fig, ax = plt.subplots(figsize=(14, 8))

for (model, returns), farbe in zip(returns_storage.items(), farben):
    cum_returns = (1 + returns).cumprod()
    ax.plot(cum_returns.index, cum_returns.values, label=model, linewidth=2, color=farbe)

baseline_cum = (1 + baseline_returns).cumprod()
ax.plot(baseline_cum.index, baseline_cum.values, label='DJIA', 
        linewidth=2, linestyle='--', color='black')

ax.set_xlabel('Datum', fontsize=12)
ax.set_ylabel('Portfolio Wert', fontsize=12)
ax.legend(loc='upper left', fontsize=10)
ax.axhline(y=1, color='gray', linestyle=':', alpha=0.5)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('kursverlauf_alle_modelle.pdf', dpi=300, bbox_inches='tight')
plt.show()

print("\n" + "="*50)
print("ENDWERTE (Portfolio-Wert nach gesamtem Zeitraum):")
print("="*50)
for model, returns in returns_storage.items():
    endwert = (1 + returns).cumprod().iloc[-1]
    rendite = (endwert - 1) * 100
    print(f"  {model:15s}: {endwert:.2f}€  (+{rendite:.1f}%)")

baseline_endwert = (1 + baseline_returns).cumprod().iloc[-1]
print(f"  {'DJIA':15s}: {baseline_endwert:.2f}€  (+{(baseline_endwert-1)*100:.1f}%)")


In [ ]:
%matplotlib inline

fig, ax = plt.subplots(figsize=(14, 8))
farben = sns.color_palette("colorblind", n_colors=len(returns_storage))

for (model, returns), farbe in zip(returns_storage.items(), farben):
    cum_returns = (1 + returns).cumprod()
    running_max = cum_returns.expanding().max()
    drawdown = (cum_returns - running_max) / running_max
    ax.fill_between(drawdown.index, 0, drawdown.values, alpha=0.3, label=model, color=farbe)
    ax.plot(drawdown.index, drawdown.values, linewidth=1, color=farbe)

baseline_cum = (1 + baseline_returns).cumprod()
baseline_running_max = baseline_cum.expanding().max()
baseline_drawdown = (baseline_cum - baseline_running_max) / baseline_running_max
ax.fill_between(baseline_drawdown.index, 0, baseline_drawdown.values, alpha=0.1, label='DJIA', color='black')
ax.plot(baseline_drawdown.index, baseline_drawdown.values, linewidth=1.5, color='black', linestyle='--', alpha=0.5)

ax.set_xlabel('Datum', fontsize=12)
ax.set_ylabel('Drawdown (%)', fontsize=12)
ax.legend(loc='lower left', fontsize=10)
ax.axhline(y=0, color='gray', linestyle='-', linewidth=0.5)
ax.grid(True, alpha=0.3)

ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.0%}'))

plt.tight_layout()
plt.savefig('drawdown_alle_modelle.pdf', dpi=300, bbox_inches='tight')
plt.show()

print("\nMaximaler Drawdown pro Modell:")
for model, returns in returns_storage.items():
    cum_returns = (1 + returns).cumprod()
    running_max = cum_returns.expanding().max()
    max_dd = ((cum_returns - running_max) / running_max).min()
    print(f"  {model:15s}: {max_dd:.2%}")

baseline_max_dd = baseline_drawdown.min()
print(f"  {'DJIA':15s}: {baseline_max_dd:.2%}")


In [ ]:
%matplotlib inline

fig, ax = plt.subplots(figsize=(12, 8))
farben = sns.color_palette("colorblind", n_colors=len(returns_storage) + 1)

scatter_data = []
for model, returns in returns_storage.items():
    annual_return = (1 + returns).cumprod().iloc[-1] ** (252/len(returns)) - 1
    annual_vol = returns.std() * np.sqrt(252)
    sharpe = annual_return / annual_vol
    scatter_data.append((model, annual_return, annual_vol, sharpe))

baseline_return = (1 + baseline_returns).cumprod().iloc[-1] ** (252/len(baseline_returns)) - 1
baseline_vol = baseline_returns.std() * np.sqrt(252)
scatter_data.append(('DJIA', baseline_return, baseline_vol, baseline_return/baseline_vol))

for i, (model, ret, vol, sharpe) in enumerate(scatter_data):
    marker = 'o' if model != 'DJIA' else 's'
    size = 200 if model != 'DJIA' else 300
    farbe = 'black' if model == 'DJIA' else farben[i]
    ax.scatter(vol, ret, s=size, c=[farbe], label=model, 
               marker=marker, edgecolors='black', linewidths=1.5)
    ax.annotate(model, (vol, ret), xytext=(10, 5), textcoords='offset points', fontsize=10)

ax.set_xlabel('Jährliche Volatilität', fontsize=12)
ax.set_ylabel('Jährliche Rendite', fontsize=12)
ax.legend(loc='lower right', fontsize=9)
ax.grid(True, alpha=0.3)

ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.0%}'))
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.0%}'))

plt.tight_layout()
plt.savefig('risk_return_scatter.pdf', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

gewichte_dateien = {
    "DDPG": "ddpg_actions.csv",
    "PPO": "ppo_actions.csv",
    "TD3": "td3_actions.csv",
    "DDPG_opt": "ddpg_Opt_actions.csv",
    "PPO_opt": "ppo_Opt_actions.csv",
    "TD3_opt": "td3_Opt_actions.csv",
    "1/N": "equal_weight_asset_allocation.csv",
    "MSRP": "markowitz_asset_allocation.csv",
}

alle_gewichte = {}
min_laenge = float('inf')

for modell, datei in gewichte_dateien.items():
    df = pd.read_csv(datei)
    
    if 'Datum' in df.columns:
        datum_spalte = pd.to_datetime(df['Datum'])
        aktien_spalten = df.columns[2:]
        df_aktien = df[aktien_spalten].copy()
        df_aktien.index = datum_spalte
    else:
        df_aktien = df.copy()
        df_aktien.index = pd.date_range(start='2023-01-04', periods=len(df), freq='B')
    
    alle_gewichte[modell] = df_aktien
    min_laenge = min(min_laenge, len(df_aktien))

for modell in alle_gewichte:
    alle_gewichte[modell] = alle_gewichte[modell].tail(min_laenge)

print(f"✅ Daten geladen: {min_laenge} Handelstage pro Modell")

n_modelle = len(gewichte_dateien)
n_cols = 2
n_rows = (n_modelle + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 5 * n_rows))
axes = axes.flatten()

aktien_farben = plt.cm.tab20(np.linspace(0, 1, 20)).tolist()
aktien_farben += plt.cm.tab20b(np.linspace(0, 1, 10)).tolist()

for idx, (modell, df) in enumerate(alle_gewichte.items()):
    ax = axes[idx]
    
    mean_gewichte = df.mean().sort_values(ascending=False)
    
    top_n = 10
    top_aktien = mean_gewichte.head(top_n).index.tolist()
    andere_aktien = mean_gewichte.tail(len(mean_gewichte) - top_n).index.tolist()
    
    df_plot = df[top_aktien].copy()
    if andere_aktien:
        df_plot['Andere'] = df[andere_aktien].sum(axis=1)
    
    spalten_sortiert = df_plot.mean().sort_values(ascending=True).index.tolist()
    df_plot = df_plot[spalten_sortiert]
    
    farben = aktien_farben[:len(df_plot.columns)]
    
    ax.stackplot(df_plot.index, 
                 [df_plot[col].values * 100 for col in df_plot.columns],
                 labels=df_plot.columns,
                 colors=farben,
                 alpha=0.8)
    
    ax.set_title(f'{modell}', fontsize=14, fontweight='bold')
    ax.set_ylabel('Gewichtung (%)', fontsize=10)
    ax.set_xlabel('')
    ax.set_ylim(0, 100)
    ax.set_xlim(df_plot.index[0], df_plot.index[-1])
    
    ax.legend(loc='upper left', fontsize=7, ncol=2, 
              bbox_to_anchor=(1.01, 1), borderaxespad=0)
    
    ax.tick_params(axis='x', rotation=45)
    ax.grid(axis='y', alpha=0.3, linestyle='--')

for idx in range(n_modelle, len(axes)):
    axes[idx].set_visible(False)

plt.tight_layout()
plt.savefig('portfolio_gewichte_stacked.pdf', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Grafik gespeichert: portfolio_gewichte_stacked.pdf")
